<a href="https://colab.research.google.com/github/mchablani/generative_ai/blob/main/pnyx/Pnyx_message_sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Read the CSV file from Google Drive
filename = '/content/drive/My Drive/pnyx/data/merged_discord_messages.csv'
df = pd.read_csv(filename)

# Ensure the CSV has a column named 'message'
if 'message' not in df.columns:
    raise ValueError("Input CSV must have a column named 'message'")

# Print the DataFrame
print(df["message"].head(10))


0                yeah use unibot or banana or shuriken
1                   Maybe I should send 1 Just in case
2                  Yeah probably unless that mfer rugs
3    isn't that bearish? it's synapse fake eth why ...
4                         i dunno i've never done that
5    does the guy even have money left to pump it f...
6                          withdraw from what? oh nice
7    yeah it is heavy don't care 500e hold for 1 mo...
8                             damn glad i insta dumped
9                               blast shitcoin airdrop
Name: message, dtype: object


In [ ]:
# Ensure all messages are strings
df['message'] = df['message'].astype(str)

In [ ]:
from transformers import pipeline
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import torch


In [ ]:
# Check if GPU is available and use it
device = 0 if torch.cuda.is_available() else -1


In [ ]:
# Load the zero-shot classification model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Define the sentiment labels
labels = ["positive", "negative", "neutral"]

# Function to classify a single sentence
def classify_sentence(sentence):
    result = classifier(sentence, candidate_labels=labels)
    return sentence, result['labels'][0], result['scores'][0]


In [ ]:
# Function to classify sentences in batches
def classify_sentences_batch(sentences):
    results = classifier(sentences, candidate_labels=labels, multi_label=False)
    return [(sentence, result['labels'][0], result['scores'][0]) for sentence, result in zip(sentences, results)]


In [ ]:
# # Perform classification in parallel
# with ThreadPoolExecutor() as executor:
#     results = list(tqdm(executor.map(classify_sentence, df['message']), total=len(df)))

# # Convert the results to a DataFrame
# result_df = pd.DataFrame(results, columns=['message', 'sentiment', 'score'])



  4%|▎         | 1273/34020 [02:02<52:30, 10.40it/s]


KeyboardInterrupt: 

In [ ]:
# Perform batched classification
batch_size = 16  # You can adjust the batch size based on your GPU memory
results = []
for i in tqdm(range(0, len(df), batch_size)):
    batch_sentences = df['message'][i:i + batch_size].tolist()
    results.extend(classify_sentences_batch(batch_sentences))




 59%|█████▊    | 1247/2127 [22:37<15:58,  1.09s/it]


KeyboardInterrupt: 

In [ ]:
# Convert the results to a DataFrame
result_df = pd.DataFrame(results, columns=['message', 'sentiment', 'score'])

In [ ]:
# Split the DataFrame into positive, negative, and neutral
positive_df = result_df[result_df['sentiment'] == 'positive']
negative_df = result_df[result_df['sentiment'] == 'negative']
neutral_df = result_df[result_df['sentiment'] == 'neutral']



In [ ]:
# Write the results to separate CSV files
positive_df.to_csv("/content/drive/My Drive/pnyx/data/positive_messages.csv", index=False)
negative_df.to_csv("/content/drive/My Drive/pnyx/data/negative_messages.csv", index=False)
neutral_df.to_csv("/content/drive/My Drive/pnyx/data/neutral_messages.csv", index=False)

In [ ]:
print(len(df))

34020


In [ ]:
print(len(positive_df))

5632


In [ ]:
print(len(negative_df))

13474


In [ ]:
print(len(neutral_df))

846
